<a href="https://colab.research.google.com/github/hoo0681/raspi4withTF/blob/master/5_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
 %tensorflow_version 2.x

import tensorflow as tf

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
print("Module Loaded.")
print("TensorFlow Version :{}".format(tf.__version__))
print("NumPy Version :{}".format(np.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

# set auto tune
AUTOTUNE = tf.data.experimental.AUTOTUNE

TensorFlow 2.x selected.
Module Loaded.
TensorFlow Version :2.1.0-rc1
NumPy Version :1.17.5
Matplotlib Version :3.1.2


In [0]:
# 지정 폴더 아래에 있는 모든 *.jpg 파일의 수
#  및 폴더명 목록을 리턴  
def check_dir(d_path):
    img_count = len(list(d_path.glob('*/*.jpg')))
    c_name = np.array([item.name for item in d_path.glob('*') if item.name != "LICENSE.txt"])
    return img_count, c_name

In [5]:
!unzip /content/drive/My\ Drive/tf2/z_data/asirra_fixed.zip

Archive:  /content/drive/My Drive/tf2/z_data/asirra_fixed.zip
replace asirra/test/cat/cat.10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
import pathlib
train_data_dir_cwd=pathlib.Path('/content/asirra/train')
test_data_dir_cwd=pathlib.Path('/content/asirra/test')

In [7]:
image_count_cwd, CLASS_NAMES_cwd = check_dir(train_data_dir_cwd)
test_image_count_cwd, CLASS_NAMES_cwd = check_dir(test_data_dir_cwd)
print('image_count: {}\nclasses: {}'.format(image_count_cwd, CLASS_NAMES_cwd))
print('image_count: {}\nclasses: {}'.format(test_image_count_cwd, CLASS_NAMES_cwd))

image_count: 12500
classes: ['dog' 'cat']
image_count: 12500
classes: ['dog' 'cat']


# 변형X 데이터

In [20]:
datagen_2=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
generator_2=datagen_2.flow_from_directory(
    directory=str(train_data_dir_cwd),
    batch_size=12500,
    shuffle=True,
    target_size=(150, 150)
)
generator_2_test=datagen_2.flow_from_directory(
    directory=str(test_data_dir_cwd),
    batch_size=125,
    shuffle=True,
    target_size=(150, 150)
)


Found 12500 images belonging to 2 classes.
Found 12500 images belonging to 2 classes.


# 변형데이터

In [0]:
train_x=np.load(file='/content/drive/My Drive/train_data_150_img.npy',allow_pickle=True)
train_y=np.load(file='/content/drive/My Drive/train_data_150_label.npy',allow_pickle=True)

In [0]:
#width_shift_range=0.3,height_shift_range=0.3,horizontal_flip=True,vertical_flip=True,rotation_range=45,zoom_range=0.2
gen_arg=tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.3,height_shift_range=0.3)
arg_train=gen_arg.flow(x=train_x,y=train_y,batch_size=12500,shuffle=True)

# 병목만들기


In [0]:
vgg_model_2=tf.keras.applications.VGG16(include_top=False,weights='imagenet',input_tensor=tf.keras.layers.Input(shape=(150,150,3)))


In [10]:
#변형없음
%%time
img,label=generator _2.next()
bottleneck_test=vgg_model_2.predict(img)

CPU times: user 1h 28min 52s, sys: 17.4 s, total: 1h 29min 10s
Wall time: 23min 9s


In [0]:
#변형있음
%%time
arg_img,arg_label=arg_train.ne 

xt()
bottleneck_arg_train=vgg_model_2.predict(arg_img)

(512, 2)

# 모델정의

In [0]:
model_bt = tf.keras.Sequential()
model_bt.add(tf.keras.layers.Flatten())
model_bt.add(tf.keras.layers.Dense(256, activation='relu'))
model_bt.add(tf.keras.layers.Dropout(0.5))
model_bt.add(tf.keras.layers.Dense(2, activation='softmax'))

model_bt.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model_bt.fit(bottleneck_test, label,
          epochs=10,
          batch_size=512,
          )

Train on 12500 samples
Epoch 1/10
12500/12500 [==============================] - 2s 161us/sample - loss: 0.7204 - accuracy: 0.7378
Epoch 2/10
12500/12500 [==============================] - 1s 107us/sample - loss: 0.3027 - accuracy: 0.8724
Epoch 3/10
12500/12500 [==============================] - 1s 106us/sample - loss: 0.2498 - accuracy: 0.8975
Epoch 4/10
12500/12500 [==============================] - 1s 106us/sample - loss: 0.2229 - accuracy: 0.9095
Epoch 5/10
12500/12500 [==============================] - 1s 107us/sample - loss: 0.2027 - accuracy: 0.9180
Epoch 6/10
12500/12500 [==============================] - 1s 107us/sample - loss: 0.1834 - accuracy: 0.9254
Epoch 7/10
12500/12500 [==============================] - 1s 107us/sample - loss: 0.1704 - accuracy: 0.9302
Epoch 8/10
12500/12500 [==============================] - 1s 106us/sample - loss: 0.1569 - accuracy: 0.9385
Epoch 9/10
12500/12500 [==============================] - 1s 106us/sample - loss: 0.1460 - accuracy: 0.9440
Epoch

In [0]:
model_bt.save_weights('/content/save_weight')

# 평가

In [0]:
vgg_model_2=tf.keras.applications.VGG16(include_top=False,weights='imagenet',input_tensor=tf.keras.layers.Input(shape=(150,150,3)))

existing_model = tf.keras.Sequential()
existing_model.add(tf.keras.layers.Flatten())
existing_model.add(tf.keras.layers.Dense(256, activation='relu'))
existing_model.add(tf.keras.layers.Dropout(0.5))
existing_model.add(tf.keras.layers.Dense(2, activation='softmax'))

existing_model.build(vgg_model_2.output_shape)

In [0]:

existing_model(vgg_model_2.output)

exvgg_model=tf.keras.Model(inputs=vgg_model_2.input,outputs=existing_model.output)



In [0]:
for layer in vgg_model_2.layers :
  layer.trainable=False

In [31]:
existing_model.load_weights('/content/save_weight')

In [0]:
exvgg_model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
exvgg_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [0]:
test_img,test_label=generator_2_test.next()

In [0]:
batch_image = np.expand_dims(test_img[1], 0)

In [36]:
exvgg_model.predict(batch_image)

array([[0.9918833 , 0.00811668]], dtype=float32)

In [37]:
exvgg_model.evaluate(generator_2_test)

  ...
    to  
  ['...']
100/100 [==============================] - 1356s 14s/step - loss: 0.2175 - accuracy: 0.9109


[0.2175437844544649, 0.91088]